This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [9]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules
from _collections import *
import json
import requests
import pprint

In [11]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
# Set up parameters
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2017-01-03'

url = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
r = requests.get(url)
data = r.json()

# Lets make things easier and let unnest dataset_data level
data = data['dataset_data']

In [12]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# pretty print this to make sense
pprint.pprint(data)

{'collapse': None,
 'column_index': None,
 'column_names': ['Date',
                  'Open',
                  'High',
                  'Low',
                  'Close',
                  'Change',
                  'Traded Volume',
                  'Turnover',
                  'Last Price of the Day',
                  'Daily Traded Units',
                  'Daily Turnover'],
 'data': [['2017-01-03',
           35.9,
           35.93,
           35.34,
           35.48,
           None,
           70618.0,
           2515473.0,
           None,
           None,
           None],
          ['2017-01-02',
           34.99,
           35.94,
           34.99,
           35.8,
           None,
           44700.0,
           1590561.0,
           None,
           None,
           None]],
 'end_date': '2017-01-03',
 'frequency': 'daily',
 'limit': None,
 'order': None,
 'start_date': '2017-01-01',
 'transform': None}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)


### 1 + 2) Collect data for the whole year

In [13]:
# Set up parameters
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2017-12-31'

url = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

# request into json and conver to dict
r = requests.get(url)
data = r.json()['dataset_data']

### 3) Highest and lowest price

There is a issue of None value in the list price. However, it is easy to work around with generator 
expression. 

There must be a faster way to get the min/max price?

In [14]:
# Open price list
open_prices = []

# the second value of each list in data.data list is the open price
for value in data.get('data'):
   open_prices.append(value[1])

# Find min and max open prices   
min_open = min(price for price in open_prices if price is not None)
max_open = max(price for price in open_prices if price is not None)

# print to console
print('The minimum open price: ', min_open,'\n')
print('The minimum open price: ', max_open)

The minimum open price:  34.0 

The minimum open price:  53.11


### 4) The largest change in one day

In [15]:
# Change list
delta = []

# Similarly as above, loop over and get the delta
for value in data.get('data'):
    delta.append(value[2] - value[3])
    
# Print max value to console
print('The maximum change in one day is:', max(delta))

The maximum change in one day is: 2.8100000000000023


### 5) The largest change in between any two days (closing price)

Simple list slicing is sufficient

In [18]:
# change list
delta = []

# Change of price based on closing
for value in data.get('data'):
    delta.append(value[4])
    
# Slice the closing price every two day
delta = [(b-a) for a, b in zip(delta[::2], delta[2::2])]

# Print the largest change
print('Largest change in stock closing price every 2 days: ',max(delta))


Largest change in stock closing price every 2 days:  3.1499999999999986


### 6, 7) Average daily trading volume during this year.

Daily traded unit is the sixth element

In [26]:
# import mean and median from statistic
from statistics import mean, median

# volume list
volume = []

# volume list
for value in data.get('data'):
    volume.append(value[6])
    
# print the average and median
print('Average volume traded: ', mean(volume))
print('Average volume traded: ', median(volume))

Average volume traded:  89124.33725490196
Average volume traded:  76286.0


### 7) Implementing own median function

Median is found by using sort and find the middle number. If list has even elements, use average 
between them to find the median.

In [49]:
def find_median(X):
    # length of X
    n = len(X)
    
    # sorted X
    X.sort()
    
    # find median   
    if n % 2 == 0:
        median_val = (X[n//2] + X[n//2 - 1])*0.5
    else:
        median_val = X[n//2]
        
    return median_val
        

In [50]:
# Test own function which gives same result
find_median(volume)






76286.0